In [1]:
### installing required packages ###
! pip install beautifultable
! pip install pandas_datareader
! pip install datetime

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 52 kB 578 kB/s 
     |████████████████████████████████| 254 kB 12.4 MB/s 


In [2]:
import pandas as pd
from pandas.tseries.offsets import DateOffset
import numpy as np
from beautifultable import BeautifulTable
import pandas_datareader as pdr
import datetime

In [3]:
### downloading the interest rate for ARM
start = datetime.datetime(1971, 4, 2)
end = datetime.datetime(2022, 7, 9)
df_rate = pdr.data.DataReader("MORTGAGE30US", 'fred', start, end)
df_rate = df_rate.reset_index()
print(df_rate)

           DATE  MORTGAGE30US
0    1971-04-02          7.33
1    1971-04-09          7.31
2    1971-04-16          7.31
3    1971-04-23          7.31
4    1971-04-30          7.29
...         ...           ...
2671 2022-06-09          5.23
2672 2022-06-16          5.78
2673 2022-06-23          5.81
2674 2022-06-30          5.70
2675 2022-07-07          5.30

[2676 rows x 2 columns]


In [4]:
def amortization_schedule(Mort_amount,Annual_rate,Years, is_arm=False, starting_date=np.nan, locked_years=0):
### CREATING PARAMETERS ###
    period_n = 0
    tot_interest_paid = 0
    interest_rate = (Annual_rate/100)/12.0
    tot_periods = Years *12
    # for arm use only:
    locked_month = locked_years * 12
    gap_month = 6
    if is_arm:
      df_rate_chunked = df_rate.loc[df_rate['DATE'] >= starting_date].reset_index()
        
### MONTHLY MORTGAGE PAYMENT ###
    if is_arm is False:
      monthly_payment = (interest_rate * Mort_amount) / ( 1 - pow(1+interest_rate,-tot_periods))

### CREATING AMORTIZATION SCHEDULE ###
    table = BeautifulTable()
    table.columns.header=["Month","Interest Rate","Payment","Principal Paydown","Interest Applied","(UPB)New Principal Balance","Total interest paid"]
    
      # Adding first row manualy
    table.rows.append([0,interest_rate, 0,0,0,Mort_amount,tot_interest_paid])

      # Adding remaining rows 
    while Mort_amount > 0:
        period_n = period_n + 1

        # update arm rates
        if is_arm: 
          if period_n <= locked_month:
            Annual_rate = df_rate_chunked['MORTGAGE30US'][0]
          elif period_n % gap_month == 0:
            curr_month = starting_date + DateOffset(months=period_n)
            curr_rate_df = df_rate_chunked.loc[df_rate_chunked['DATE'] >= curr_month].copy().reset_index()
            if curr_rate_df.shape[0]>0:
              Annual_rate = df_rate_chunked.loc[df_rate_chunked['DATE'] >= curr_month].copy().reset_index()['MORTGAGE30US'][0]
          interest_rate = (Annual_rate/100)/12.0
          monthly_payment = (interest_rate * Mort_amount) / ( 1 - pow(1+interest_rate,-(tot_periods-period_n+1)))

        monthly_interest = (Mort_amount * interest_rate)
        monthly_principal = monthly_payment - monthly_interest
        tot_interest_paid = tot_interest_paid + monthly_interest
        if Mort_amount - monthly_payment < 0:
          monthly_principal = Mort_amount
        table.rows.append([period_n,Annual_rate,round(monthly_payment,2),round(monthly_principal,2),round(monthly_interest,2),round(Mort_amount-monthly_principal,2),round(tot_interest_paid,2)])
        Mort_amount = Mort_amount - monthly_principal
    print(f'Amortization schedule for {Years} year 1000000 $ mortgage structured with a monthly payment of {round(monthly_payment,2)}$ and an annual interest rate of {Annual_rate}%.\n',table)

In [5]:
amortization_schedule(1000000,4,30)

Amortization schedule for 30 year 1000000 $ mortgage structured with a monthly payment of 4774.15$ and an annual interest rate of 4%.
 +-------+--------+---------+-----------+----------+----------------+-----------+
| Month | Intere | Payment | Principal | Interest | (UPB)New Princ | Total int |
|       | st Rat |         |  Paydown  |  Applied |  ipal Balance  | erest pai |
|       |   e    |         |           |          |                |     d     |
+-------+--------+---------+-----------+----------+----------------+-----------+
|   0   | 0.003  |    0    |     0     |    0     |    1000000     |     0     |
+-------+--------+---------+-----------+----------+----------------+-----------+
|   1   |   4    | 4774.15 |  1440.82  | 3333.33  |   998559.18    |  3333.33  |
+-------+--------+---------+-----------+----------+----------------+-----------+
|   2   |   4    | 4774.15 |  1445.62  | 3328.53  |   997113.56    |  6661.86  |
+-------+--------+---------+-----------+----------+----

In [6]:
amortization_schedule(1000000,2.5,20)

Amortization schedule for 20 year 1000000 $ mortgage structured with a monthly payment of 5299.03$ and an annual interest rate of 2.5%.
 +-------+--------+---------+-----------+----------+----------------+-----------+
| Month | Intere | Payment | Principal | Interest | (UPB)New Princ | Total int |
|       | st Rat |         |  Paydown  |  Applied |  ipal Balance  | erest pai |
|       |   e    |         |           |          |                |     d     |
+-------+--------+---------+-----------+----------+----------------+-----------+
|   0   | 0.002  |    0    |     0     |    0     |    1000000     |     0     |
+-------+--------+---------+-----------+----------+----------------+-----------+
|   1   |  2.5   | 5299.03 |  3215.7   | 2083.33  |    996784.3    |  2083.33  |
+-------+--------+---------+-----------+----------+----------------+-----------+
|   2   |  2.5   | 5299.03 |  3222.39  | 2076.63  |   993561.91    |  4159.97  |
+-------+--------+---------+-----------+----------+--

In [7]:
amortization_schedule(1000000,0.0,30, is_arm=True, starting_date=datetime.datetime(1992, 1, 1), locked_years=7)

Amortization schedule for 30 year 1000000 $ mortgage structured with a monthly payment of 5626.37$ and an annual interest rate of 3.22%.
 +-------+--------+---------+-----------+----------+----------------+-----------+
| Month | Intere | Payment | Principal | Interest | (UPB)New Princ | Total int |
|       | st Rat |         |  Paydown  |  Applied |  ipal Balance  | erest pai |
|       |   e    |         |           |          |                |     d     |
+-------+--------+---------+-----------+----------+----------------+-----------+
|   0   |  0.0   |    0    |     0     |    0     |    1000000     |     0     |
+-------+--------+---------+-----------+----------+----------------+-----------+
|   1   |  8.24  | 7505.64 |  638.97   | 6866.67  |   999361.03    |  6866.67  |
+-------+--------+---------+-----------+----------+----------------+-----------+
|   2   |  8.24  | 7505.64 |  643.36   | 6862.28  |   998717.67    | 13728.95  |
+-------+--------+---------+-----------+----------+-